In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt



In [6]:
import os

# Define the path to the OneDrive folder
onedrive_path = os.path.expanduser('/Users/ankitkumar/Library/CloudStorage/OneDrive-GeorgeMasonUniversity-O365Production/HackFax/')

# Define the path to the specific file within the OneDrive folder
file_path = os.path.join(onedrive_path, 'samplegeocodiooutput_acseconomics.csv')

# Read the data into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,id,name,address,city,state,zip_code,Latitude,Longitude,Accuracy Score,Accuracy Type,...,Census Block Code,Census Block Group,Full FIPS,Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code
0,18436,Peregrine Espresso,660 Pennsylvania Ave SE,Washington,DC,20003,38.885172,-76.996565,1.0,rooftop,...,1021,1,110010066001021,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47900,metropolitan,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47894.0
1,18437,Peregrine Espresso,1718 14th St NW,Washington,DC,20009,38.913274,-77.032266,1.0,rooftop,...,4000,4,110010052014000,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47900,metropolitan,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47894.0
2,18438,Peregrine Espresso,1309 5th St NE,Washington,DC,20002,38.908724,-76.997653,1.0,rooftop,...,1019,1,110010088031019,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47900,metropolitan,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47894.0
3,18439,Soho Tea and Coffee,2150 P St NW,Washington,DC,20037,38.909480,-77.048527,1.0,rooftop,...,4000,4,110010055004000,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47900,metropolitan,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47894.0
4,18440,Ebenezers Coffeehouse,201 F Street NE,Washington,DC,20002,38.897139,-77.003286,1.0,rooftop,...,2003,2,110010083012003,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47900,metropolitan,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",47894.0
